<a href="https://colab.research.google.com/github/Haji-Ai-maker/Haji-Ai-maker/blob/main/quantum_mnist_classification/Classifying_mnist_data_using_quantum_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will first install Qulacs plugin with GPU for Pennylane and then refresh the environment.

In [ ]:
import os
!pip install git+https://github.com/kareem1925/pennylane-qulacs@GPU_support
os.kill(os.getpid(), 9)

  Cloning https://github.com/kareem1925/pennylane-qulacs (to revision GPU_support) to /tmp/pip-req-build-clb7771c
  Running command git clone -q https://github.com/kareem1925/pennylane-qulacs /tmp/pip-req-build-clb7771c
  Running command git checkout -b GPU_support --track origin/GPU_support
  Switched to a new branch 'GPU_support'
  Branch 'GPU_support' set up to track remote branch 'GPU_support' from 'origin'.
     |████████████████████████████████| 204kB 9.4MB/s 
     |████████████████████████████████| 276kB 68.2MB/s 
  Created wheel for pennylane-qulacs: filename=pennylane_qulacs-0.0.4-cp36-none-any.whl size=4565 sha256=a19761f9c721e874da6a48eeda931d642fd6cd4e972eb89396f11682c41c42cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-eb9omc4u/wheels/93/58/04/19f31c7d05f27562d0941ae63f8c34e73620017d13840d4d00
  Created wheel for qulacs-gpu: filename=Qulacs_GPU-0.1.9-cp36-cp36m-linux_x86_64.whl size=843504 sha256=ad478d3e31973ec60d0475e83bf33a1f2ce037bcabce4e3162bef676ceeb52b4
  Store

Run the following command to make sure everything is working perfectly

In [ ]:
import qulacs
qulacs.QuantumStateGpu

qulacs.QuantumStateGpu

In [ ]:
from pennylane import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,normalize,LabelBinarizer
from sklearn.utils import compute_sample_weight
import pennylane as qml
from sklearn.datasets import load_digits
import warnings
from sklearn.metrics import balanced_accuracy_score as acc
from pennylane.optimize import AdamOptimizer,AdagradOptimizer

np.seterr(all="ignore")
warnings.filterwarnings('ignore')

**Defining the log loss function along with softmax and accuracy**

In [ ]:

# this function is taken from scikit-learn code of meterics and it has been
# modified to comply to autograd's numpy

def log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None,
             labels=None):


    lb = LabelBinarizer()

    if labels is not None:
        lb.fit(labels)
    else:
        lb.fit(y_true)

    if len(lb.classes_) == 1:
        if labels is None:
            raise ValueError('y_true contains only one label ({0}). Please '
                             'provide the true labels explicitly through the '
                             'labels argument.'.format(lb.classes_[0]))
        else:
            raise ValueError('The labels array needs to contain at least two '
                             'labels for log_loss, '
                             'got {0}.'.format(lb.classes_))

    transformed_labels = lb.transform(y_true)

    if transformed_labels.shape[1] == 1:
        transformed_labels = np.append(1 - transformed_labels,
                                       transformed_labels, axis=1)

    # Clipping
    y_pred = np.clip(y_pred, eps, 1 - eps)

    # If y_pred is of single dimension, assume y_true to be binary
    # and then check.
    if y_pred.ndim == 1:
        y_pred = y_pred[:, np.newaxis]
    if y_pred.shape[1] == 1:
        y_pred = np.append(1 - y_pred, y_pred, axis=1)

    # Check if dimensions are consistent.
#    transformed_labels = check_array(transformed_labels)
    if len(lb.classes_) != y_pred.shape[1]:
        if labels is None:
            raise ValueError("y_true and y_pred contain different number of "
                             "classes {0}, {1}. Please provide the true "
                             "labels explicitly through the labels argument. "
                             "Classes found in "
                             "y_true: {2}".format(transformed_labels.shape[1],
                                                  y_pred.shape[1],
                                                  lb.classes_))
        else:
            raise ValueError('The number of classes in labels is different '
                             'from that in y_pred. Classes found in '
                             'labels: {0}'.format(lb.classes_))

    # Renormalize
    y_pred /= y_pred.sum(axis=1)[:, np.newaxis]
    loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
#    print(loss)

    return loss


def accuracy_score(y_true, y_pred):
    """

    This function computed the weighted aaverage accuarcy

    """
    weights = compute_sample_weight('balanced',y_true)

    return acc(y_true,y_pred,sample_weight=weights)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""

    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


**Data loading and splitting**

In [ ]:
X,y = load_digits(n_class=3,return_X_y=True)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=5,stratify=y)

la  = OneHotEncoder(sparse=False).fit(y.reshape(-1,1))

y_train = la.transform(y_train.reshape(-1,1))

y_test = la.transform(y_test.reshape(-1,1))

y_test[:2]

array([[0., 0., 1.],
       [1., 0., 0.]])

**Defining the quantum circuit**

In [ ]:
# initialize the device
dev = qml.device("qulacs.simulator", wires=7,shots=1000,analytic = True)

@qml.qnode(dev)
def qclassifier(weights, X=None):


    # pennylane normalizes the input for us by setting normalize to True so no need for any preprocessing
    qml.templates.AmplitudeEmbedding(X,wires=list(range(7)),pad=0.0,normalize=True)

### the following comments mimics the same line below except for CRX gate where you should define its weights
### because the init template in pennylane doesn't do that it only initializes the rotation parameters

#    for i in range(weights.shape[0]):
 #     for j in range(weights.shape[1]):
  #      qml.Rot(*weights[i][j],wires=j)
   #   for x in range(cnots.shape[1]):
    #    qml.CRX(*cnots[i][x],wires=[x,(x+1)%6])


    qml.templates.StronglyEntanglingLayers(weights,wires=list(range(7)))

    return [qml.expval(qml.PauliZ(i)) for i in range(7)]

### **The Cost Function**
This fucntion contains the main logic of the full network. it takes a batched input with the weights and first pass the the quantum weights into the quantum classifier.
Then it adds the bias to the output from Qcircuit. After that, we apply the classical operations the relu and softmax as shown in the for loop below.

In [1]:


def cost(params, x, y):


    # Compute prediction for each input in data batch
    loss = []
    for i in range(len(x)):

      out =  qclassifier(params[0],X=x[i])+params[1]              # quantum output
      out = np.maximum(0,np.dot(params[2],out)+params[3])         # reul on the first layer
      loss.append(softmax(np.dot(params[4],out)+params[5]))       # softmax on the second layer
    loss = log_loss(y,np.array(loss),labels=y_train)              # compute loss

    weights = compute_sample_weight('balanced',y)

    # weighted average to compensate for imbalanced batches
    s = 0
    for x, y in zip(loss, weights):
      s += x * y

    return s/sum(weights)


In [1]:

# a helper function to predict the label of the image

def predict(params,x,y):
  prob = []
  for i in range(len(x)):
    out = qclassifier(params[0],X=x[i])+params[1]
    out = np.maximum(0,np.dot(params[2],out)+params[3])
    out = softmax(np.dot(params[4],out)+params[5])
    prob.append(np.argmax(out))
  return prob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Weights initialization**

In [4]:
np.random.seed(88)
# quantum parameters
n_qubits= 7
Q_n_layer = 8

Qweights = qml.init.strong_ent_layers_uniform(n_layers = Q_n_layer,n_wires = n_qubits,low=0,high=1,seed=0)
Qbias = np.random.uniform(low=-.1,high=.1,size=(n_qubits))*0
# first layer parameters
hidden_units = 12
linear2_layer = np.random.randn(hidden_units,n_qubits)*0.01
bias2_layer = np.random.randn(hidden_units)*0

classes = 3
# second layer parameters
linear3_layer = np.random.randn(classes,hidden_units)*0.01
bias3_layer = np.random.randn(classes)*0

params = [Qweights,Qbias,linear2_layer,bias2_layer,linear3_layer,bias3_layer]
params

NameError: name 'np' is not defined

**Load the saved weights**

You can download the weights from this [link](https://github.com/kareem1925/Ismailia-school-of-AI/raw/master/quantum_mnist_classification/final-grads.npy). Or, you can check the [repo](https://github.com/kareem1925/Ismailia-school-of-AI/tree/master/quantum_mnist_classification) itself.

In [3]:
final_weights = np.load("C:\Users\PCIS-043\Downloads\AIML PROJECT 4\visa_Approval\VISA_APPROVAL_ipynb - Colab_files",allow_pickle=True)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-3-18cadc635ceb>, line 1)

Convert the one hot encoding back to its original labels

In [6]:
labels = la.inverse_transform(y_test)

NameError: name 'la' is not defined

In [ ]:
predictions=predict(final_weights,X_test,y_test)

In [ ]:
print(accuracy_score(labels,predictions))

1.0

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels,predictions))

### **Training procedure**
you can run this cell and have fun with the training.

In [12]:
from sklearn.utils import shuffle

learning_rate = 0.12
epochs = 1200
batch_size = 32

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# classical adam optimizer
loss = np.inf #random large number
grads = []


for it in range(epochs):

    # data shuffling
    X_train_1,y_train_1 = shuffle(X_train,y_train)
    X_test_1,y_test_1 = shuffle(X_test,y_test)

    # batching the data, i.e. every epoch processes the batch_size samples only
    Xbatch = X_train_1[:batch_size]
    ybatch = y_train_1[:batch_size]

    params = opt.step(lambda v: cost(v, Xbatch, ybatch), params)    # updating weights


    grads.append(params)

    if it % 1 == 0:

      test_loss = cost(params, X_test_1[:50], y_test_1[:50])
      if test_loss < loss:
        loss = test_loss
        print('heey new loss')
      print("Iter: {:5d} | test loss: {:0.7f} ".format(it + 1, test_loss))

NameError: name 'tf' is not defined